<a href="https://colab.research.google.com/github/pratimdas/googlecolab/blob/main/Chapter_2_Recipe_14_Excel_Header_%26_Multi_Sheet_Handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recipe 14: Excel Header & Multi-Sheet Handling
Using /content/sample_data/retail_store_sales-xlsx.xlsx with “online” and “in-store” sheets

**Purpose**:
Learn to load Excel workbooks with multiple sheets, handle extra header rows or merged-cell layouts, and combine them into a single DataFrame for unified analysis.


# Step 1: Inspect the workbook and its sheets

In [ ]:
#Step 1: Load Inspect the workbook and its sheets
import pandas as pd

# Path to the Excel file
filepath = '/content/sample_data/retail_store_sales-xlsx.xlsx'

# Load workbook metadata
xls = pd.ExcelFile(filepath, engine='openpyxl')
print("Available sheets:", xls.sheet_names)


Available sheets: ['online', 'in-store']


In [ ]:
# Step 2: Read each sheet with proper header handling
# If the first row(s) contain metadata or merged headers, use skiprows to bypass them.
# Here we assume the actual column names are on the first row (index 0), so no skip:
df_online = pd.read_excel(
    filepath,
    sheet_name='online',
    header=0,        # row 0 contains column names
    engine='openpyxl'
)
df_instore = pd.read_excel(
    filepath,
    sheet_name='in-store',
    header=0,
    engine='openpyxl'
)

print("Online sheet shape: ", df_online.shape)
print("In-Store sheet shape:", df_instore.shape)


Online sheet shape:  (6354, 11)
In-Store sheet shape: (6221, 11)


In [ ]:
# Step 3: Handle merged cells or propagated labels
# If any columns were merged in Excel (leading to NaNs in pandas), forward-fill them:
for df in (df_online, df_instore):
    df.fillna(method='ffill', axis=0, inplace=True)
print("Merged-cell labels forward-filled where needed.")

# Step 4: Tag and combine sheets
# Add a column to retain the source sheet
df_online['Source']  = 'online'
df_instore['Source'] = 'in-store'

# Concatenate into one DataFrame
df_full = pd.concat([df_online, df_instore], ignore_index=True)
print("Combined DataFrame shape:", df_full.shape)

# Step 5: Quick verification
# Show counts by source to verify both sheets loaded
print(df_full['Source'].value_counts())

# Preview combined data
df_full.head(5)


Merged-cell labels forward-filled where needed.
Combined DataFrame shape: (12575, 12)
Source
online      6354
in-store    6221
Name: count, dtype: int64


/tmp/ipython-input-4-873023163.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', axis=0, inplace=True)


,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied,Source
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,1.0,online
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,1.0,online
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,0.0,online
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,0.0,online
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,0.0,online
